# ห้องปฏิบัติการเสริม: การปรับขนาดคุณสมบัติ (Feature scaling) และอัตราการเรียนรู้ (Learning Rate) (หลายตัวแปร)

เป้าหมาย
ในห้องปฏิบัติการนี้ คุณจะ:

- ใช้รูทีนการคำนวณตัวแปรหลายตัวที่พัฒนาในห้องปฏิบัติการก่อนหน้านี้
- รันอัลกอริทึม Gradient Descent บนชุดข้อมูลที่มีหลายฟีเจอร์
- สำรวจผลกระทบของอัตราการเรียนรู้ (learning rate alpha) ต่ออัลกอริทึม Gradient Descent
- เพิ่มประสิทธิภาพของอัลกอริทึม Gradient Descent โดยใช้การปรับขนาดฟีเจอร์ (feature scaling) ด้วยวิธี z-score = normalization

## เครื่องมือ
คุณจะใช้ฟังก์ชันที่พัฒนาในแล็บสุดท้าย รวมถึง matplotlib และ NumPy

In [ ]:
# prompt: import requsts and download from this github link : https://raw.githubusercontent.com/Smith-WeStrideTH/Machine_Learning_Course/main/work/deeplearning.mplstyle

import requests
from pathlib import Path

url = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Machine_Learning_Course/main/work/deeplearning.mplstyle'
url2 = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Machine_Learning_Course/main/work/lab_utils_multi.py'
url3 = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Machine_Learning_Course/main/work/data/houses.txt'
url4 = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Machine_Learning_Course/main/work/lab_utils_common.py'

response = requests.get(url)
with open('deeplearning.mplstyle', 'wb') as f:
  f.write(response.content)


response = requests.get(url2)
with open('lab_utils_multi.py', 'wb') as f:
  f.write(response.content)

response = requests.get(url3)
data_path = Path('data')
if not data_path.is_dir():
  data_path.mkdir(parents=True, exist_ok=True)
with open('data/houses.txt', 'wb') as f:
  f.write(response.content)

response = requests.get(url4)
with open('lab_utils_common.py', 'wb') as f:
  f.write(response.content)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lab_utils_multi import  load_house_data, run_gradient_descent 
from lab_utils_multi import  norm_plot, plt_equal_scale, plot_cost_i_w
from lab_utils_common import dlc
np.set_printoptions(precision=2)
plt.style.use('./deeplearning.mplstyle')

## สัญลักษณ์และความหมาย:

|General <br />  Notation  | Description| Python (if applicable) |
|: ------------|: ------------------------------------------------------------||
| $a$ | สเกลาร์ (scalar) ไม่เป็นตัวหนา                                                      ||
| $\mathbf{a}$ | เวกเตอร์ (vector) เป็นตัวหนา                                                 ||
| $\mathbf{A}$ | เมทริกซ์ (matrix) เป็นตัวหนาและตัวใหญ่                                         ||
| **Regression** |         |    |     |
|  $\mathbf{X}$ | เมทริกซ์ของตัวอย่างการฝึก (training examples)                  | `X_train` |   
|  $\mathbf{y}$  | เวกเตอร์ของเป้าหมาย (targets) ของตัวอย่างการฝึก                | `y_train` 
|  $\mathbf{x}^{(i)}$, $y^{(i)}$ | $i_{th}$ตัวอย่างการฝึกตัวที่ i | `X[i]`, `y[i]`|
| m | จำนวนตัวอย่างการฝึก	 | `m`|
| n | จำนวนคุณลักษณะ (features) ในแต่ละตัวอย่าง	 | `n`|
|  $\mathbf{w}$  |  พารามิเตอร์: น้ำหนัก (weight)	,                       | `w`    |
|  $b$           |  พารามิเตอร์: ไบแอส (bias)	                                           | `b`    |     
| $f_{\mathbf{w},b}(\mathbf{x}^{(i)})$ | ผลลัพธ์ของการประเมินโมเดลที่ตัวอย่าง  $\mathbf{x}^{(i)}$  ด้วยพารามิเตอร์ $\mathbf{w},b$: $f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = \mathbf{w} \cdot \mathbf{x}^{(i)}+b$  | `f_wb` | 
|$\frac{\partial J(\mathbf{w},b)}{\partial w_j}$| เกรเดียนต์ (gradient) หรืออนุพันธ์ย่อยของค่าใช้จ่าย (cost) เทียบกับพารามิเตอร์  $w_j$ |`dj_dw[j]`| 
|$\frac{\partial J(\mathbf{w},b)}{\partial b}$| เกรเดียนต์ (gradient) หรืออนุพันธ์ย่อยของค่าใช้จ่าย (cost) เทียบกับพารามิเตอร์  $b$| `dj_db`|

#  ภารกิจ: คาดการณ์ราคาบ้าน
เหมือนกับ Lab ก่อนๆ คุณจะใช้ตัวอย่างการคาดการณ์ราคาบ้านเป็นแรงบันดาลใจ

ชุดข้อมูลฝึกประกอบด้วยตัวอย่างหลายตัวที่มี 4 คุณลักษณะ (ขนาด, ห้องนอน, ชั้น และอายุ) ตามตารางด้านล่าง โปรดสังเกตว่าใน Lab นี้ คุณลักษณะขนาดเป็นหน่วย sqft ในขณะที่ Lab ก่อนๆ ใช้หน่วย 1000 sqft ชุดข้อมูลนี้มีขนาดใหญ่กว่า Lab ก่อนๆ

เราต้องการสร้างโมเดลการถดถอยเชิงเส้น (Linear Regression Model) โดยใช้ค่าเหล่านี้ เพื่อที่เราจะสามารถคาดการณ์ราคาของบ้านหลังอื่นๆ ได้ เช่น บ้านขนาด 1200 sqft มี 3 ห้องนอน 1 ชั้น และอายุ 40 ปี


##  Dataset: 
| Size (sqft) | Number of Bedrooms  | Number of floors | Age of  Home | Price (1000s dollars)  |   
| ----------------| ------------------- |----------------- |--------------|----------------------- |  
| 952             | 2                   | 1                | 65           | 271.5                  |  
| 1244            | 3                   | 2                | 64           | 232                    |  
| 1947            | 3                   | 2                | 17           | 509.8                  |  
| ...             | ...                 | ...              | ...          | ...                    |


In [ ]:
# load the dataset
X_train, y_train = load_house_data()
X_features = ['size(sqft)','bedrooms','floors','age']

มาดูข้อมูลและลักษณะของข้อมูลโดยการพล็อตแต่ละลักษณะกับราคา

In [ ]:
fig,ax=plt.subplots(1, 4, figsize=(12, 3), sharey=True)
for i in range(len(ax)):
    ax[i].scatter(X_train[:,i],y_train)
    ax[i].set_xlabel(X_features[i])
ax[0].set_ylabel("Price (1000's)")
plt.show()

การพล็อตแต่ละคุณลักษณะเทียบกับเป้าหมายคือราคา จะแสดงให้เห็นถึงคุณลักษณะใดที่มีอิทธิพลต่อราคาอย่างมากที่สุด จากภาพข้างบน การเพิ่มขนาดก็จะเพิ่มราคาด้วย ห้องนอนและชั้นดูเหมือนจะไม่มีผลกระทบอย่างมากต่อราคา บ้านใหม่มีราคาสูงกว่าบ้านเก่า

<a name="toc_15456_5"></a>
## Gradient Descent With Multiple Variables - การไล่ระดับลงด้วยตัวแปรหลายตัว
นี่คือสมการที่คุณได้พัฒนาขึ้นในห้องปฏิบัติการสุดท้ายเกี่ยวกับการไล่ระดับลงสำหรับตัวแปรหลายตัว:

$$\begin{align*} \text{ทำซ้ำ}&\text{ จนกว่า convergence:} \; \lbrace \newline\;
& w_j := w_j -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial w_j} \tag{1}  \; & \text{for j = 0..n-1}\newline
&b\ \ := b -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial b}  \newline \rbrace
\end{align*}$$

โดยที่ n คือจำนวนลักษณะเฉพาะ พารามิเตอร์ $w_j$,  $b$, จะได้รับการอัปเดตพร้อมกัน และที่  

$$
\begin{align}
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})x_{j}^{(i)} \tag{2}  \\
\frac{\partial J(\mathbf{w},b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)}) \tag{3}
\end{align}
$$
* m คือจำนวนตัวอย่างการฝึกอบรมในชุดข้อมูล

    
*  $f_{\mathbf{w},b}(\mathbf{x}^{(i)})$ คือการทำนายของโมเดล ในขณะที่ $y^{(i)}$ คือค่าเป้าหมาย


## Learning Rate
<figure>
    <img src="./images/C1_W2_Lab06_learningrate_v1.png" style="width:1200px;" >
</figure>
  
การบรรยายได้กล่าวถึงปัญหาบางประการที่เกี่ยวข้องกับการตั้งอัตราการเรียนรู้  $\alpha$ อัตราการเรียนรู้ควบคุมขนาดของการปรับปรุงพารามิเตอร์ ดูสมการ (1) ด้านบน อัตราการเรียนรู้จะถูกแบ่งปันโดยพารามิเตอร์ทั้งหมด

มาลองรันการไล่ระดับสีและลองใช้ค่า  $\alpha$ บางค่ากับชุดข้อมูลของเรา


### $\alpha$ = 9.9e-7

In [ ]:
#set alpha to 9.9e-7
_, _, hist = run_gradient_descent(X_train, y_train, 10, alpha = 9.9e-7)

ดูเหมือนอัตราการเรียนรู้จะสูงเกินไป ผลลัพธ์ไม่ลู่เข้า ค่าใช้จ่าย เพิ่มขึ้น แทนที่จะลดลง มาพล็อตผลลัพธ์กัน:

In [ ]:
plot_cost_i_w(X_train, y_train, hist)

กราฟทางขวาแสดงค่าของหนึ่งในพารามิเตอร์คือ  $w_0$ ในแต่ละรอบการทำซ้ำ ค่าจะเกินค่าที่เหมาะสม ซึ่งส่งผลให้ค่าใช้จ่ายเพิ่มขึ้นแทนที่จะเข้าใกล้ค่าต่ำสุด โปรดทราบว่านี่ไม่ใช่ภาพที่สมบูรณ์เนื่องจากมีการปรับเปลี่ยนพารามิเตอร์ 4 ตัวในแต่ละรอบแทนที่จะเป็นเพียงตัวเดียว กราฟนี้แสดงเฉพาะ  $w_0$ โดยที่พารามิเตอร์อื่น ๆ ถูกตรึงไว้ที่ค่าที่ไม่เป็นอันตราย ในกราฟนี้และกราฟต่อไป คุณอาจสังเกตเห็นเส้นสีน้ำเงินและสีส้มเบี่ยงเบนเล็กน้อย


### $\alpha$ = 9e-7
ลองใช้ค่าที่เล็กกว่าดูว่าจะเกิดอะไรขึ้น

In [ ]:
#set alpha to 9e-7
_,_,hist = run_gradient_descent(X_train, y_train, 10, alpha = 9e-7)

Cost ลดลงตลอดการรัน แสดงว่า alpha ไม่ใหญ่เกินไป

In [ ]:
plot_cost_i_w(X_train, y_train, hist)

ทางซ้าย คุณจะเห็นว่าต้นทุนลดลงตามที่ควรจะเป็น ทางขวา คุณจะเห็นว่า $w_0$ 
ยังคงแกว่งไปมารอบ ๆ ค่าต่ำสุด แต่ต้นทุนลดลงในแต่ละรอบ ไม่ได้เพิ่มขึ้น โปรดสังเกตข้างต้นว่า `dj_dw[0]` เปลี่ยนเครื่องหมายในแต่ละรอบ เนื่องจาก `w[0]` กระโดดข้ามค่าที่เหมาะสม ค่าอัลฟา (alpha) นี้จะลู่เข้า คุณสามารถเปลี่ยนจำนวนรอบเพื่อดูพฤติกรรมของมันได้

### $\alpha$ = 1e-7
ลองใช้ค่า $\alpha$  ที่เล็กลงเล็กน้อย แล้วดูว่าจะเกิดอะไรขึ้น

In [ ]:
#set alpha to 1e-7
_,_,hist = run_gradient_descent(X_train, y_train, 10, alpha = 1e-7)

ลดลงตลอดการรัน แสดงให้เห็นว่า $\alpha$ ไม่ใหญ่เกินไป 


In [ ]:
plot_cost_i_w(X_train,y_train,hist)

ด้านซ้าย คุณจะเห็นว่าต้นทุนกำลังลดลงตามที่ควรจะเป็น ด้านขวา คุณจะเห็นว่า $w_0$ กำลังเข้าใกล้ค่าต่ำสุดโดยไม่มีการแกว่ง `dj_w0` มีค่าติดลบตลอดการทำงาน โซลูชันนี้จะลู่เข้าด้วย

## Feature Scaling 
<figure>
    <img src="./images/C1_W2_Lab06_featurescalingheader.PNG" style="width:1200px;" >
</figure>
บรรยายในหลักสูตรเน้นความสำคัญของการปรับขนาดข้อมูลเพื่อให้คุณลักษณะมีช่วงใกล้เคียงกัน
หากคุณสนใจรายละเอียดว่าทำไมจึงเป็นเช่นนั้น ให้คลิกหัวข้อ 'Details' ด้านล่าง หากไม่สนใจ ส่วนด้านล่างจะแนะนำการใช้งานการปรับขนาดคุณลักษณะ

<details>
<summary>
    <font size='3', color='darkgreen'><b>Details</b></font>
</summary>

มาดูสถานการณ์อีกครั้งกับ $\alpha$ = 9e-7 นี่ใกล้เคียงกับค่าสูงสุดที่เราสามารถตั้งค่า $\alpha$ ได้โดยไม่เกิดการแยกตัว นี่คือการรันสั้น ๆ แสดงการวนซ้ำครั้งแรก ๆ:

<figure>
    <img src="./images/C1_W2_Lab06_ShortRun.PNG" style="width:1200px;" >
</figure>


จากข้างต้น ในขณะที่ค่าใช้จ่ายลดลง เห็นได้ชัดว่า $w_0$ กำลังก้าวหน้าเร็วกว่าพารามิเตอร์อื่น ๆ เนื่องจากมีเกรเดียนต์ที่ใหญ่กว่ามาก

กราฟด้านล่างแสดงผลลัพธ์ของการรันที่ยาวนานมากด้วย  $\alpha$ = 9e-7 ซึ่งใช้เวลาหลายชั่วโมง
<figure>
    <img src="./images/C1_W2_Lab06_LongRun.PNG" style="width:1200px;" >
</figure>

จากข้างบน คุณสามารถเห็นต้นทุนลดลงอย่างช้าๆ หลังจากการลดลงครั้งแรก สังเกตความแตกต่างระหว่าง `w0` และ `w1`,`w2`,`w3` เช่นเดียวกับ `dj_dw0` และ `dj_dw1-3` `w0` เข้าใกล้ค่าสุดท้ายได้อย่างรวดเร็ว และ `dj_dw0` ลดลงอย่างรวดเร็วเป็นค่าเล็กๆ แสดงว่า `w0` ใกล้เคียงกับค่าสุดท้าย พารามิเตอร์อื่นๆ ลดลงอย่างช้ากว่ามาก"

เหตุใดจึงเป็นเช่นนี้? มีสิ่งใดที่เราสามารถปรับปรุงได้หรือไม่? ดูด้านล่าง:
<figure>
    <center> <img src="./images/C1_W2_Lab06_scale.PNG"   ></center>
</figure>   

ภาพด้านบนแสดงให้เห็นว่าเหตุใด w จึงได้รับการอัปเดตอย่างไม่สม่ำเสมอ

- $\alpha$  ถูกแชร์โดยการอัปเดตพารามิเตอร์ทั้งหมด ($w$'s and $b$)
- ค่าความผิดพลาดทั่วไปจะถูกคูณด้วยลักษณะเฉพาะสำหรับ $w$' (ไม่ใช่ $b$)
- ลักษณะเฉพาะมีความแตกต่างกันอย่างมากในขนาด ทำให้บางลักษณะได้รับการอัปเดตเร็วกว่าลักษณะอื่นมาก ในกรณีนี้ $w_0$ ถูกคูณด้วย 'size(sqft)' ซึ่งโดยทั่วไป > 1000 ในขณะที่ $w_1$ ถูกคูณด้วย 'จำนวนห้องนอน' ซึ่งโดยทั่วไปคือ 2-4

วิธีแก้ไขคือการปรับมาตราส่วนของลักษณะเฉพาะ"

The lectures discussed three different techniques: 
- การปรับขนาดลักษณะเฉพาะ (feature scaling) โดยทั่วไปจะแบ่งแต่ละลักษณะเฉพาะที่เป็นบวกด้วยค่าสูงสุดของมัน หรือโดยทั่วไปกว่านั้น จะปรับขนาดแต่ละลักษณะเฉพาะโดยใช้ทั้งค่าต่ำสุดและสูงสุดของมัน โดยใช้สูตร (x-min)/(max-min)ทั้งสองวิธีทำให้ลักษณะเฉพาะเป็นมาตรฐานอยู่ในช่วง -1 ถึง 1 โดยวิธีแรกใช้ได้กับลักษณะเฉพาะที่เป็นบวก ซึ่งง่ายและเหมาะสำหรับตัวอย่างในบรรยาย และวิธีที่สองใช้ได้กับลักษณะเฉพาะใด ๆ
- Mean normalization: $x_i := \dfrac{x_i - \mu_i}{max - min} $ 
- Z-score normalization ซึ่งเราจะสำรวจด้านล่าง


### การทำให้เป็นมาตรฐาน z-score
หลังจากการทำให้เป็นมาตรฐาน z-score คุณสมบัติทั้งหมดจะมีค่าเฉลี่ยเป็น 0 และส่วนเบี่ยงเบนมาตรฐานเป็น 1

เพื่อดำเนินการทำให้เป็นมาตรฐาน z-score ให้ปรับค่าอินพุตของคุณตามสูตรนี้:

$$x^{(i)}_j = \dfrac{x^{(i)}_j - \mu_j}{\sigma_j} \tag{4}$$ 
โดยที่ $j$ เลือกคุณสมบัติหรือคอลัมน์ในเมทริกซ์ $\mathbf{X}$ . $µ_j$ คือค่าเฉลี่ยของค่าทั้งหมดสำหรับคุณสมบัติ  (j) และ $\sigma_j$ คือส่วนเบี่ยงเบนมาตรฐานของคุณสมบัติ (j).
$$
\begin{align}
\mu_j &= \frac{1}{m} \sum_{i=0}^{m-1} x^{(i)}_j \tag{5}\\
\sigma^2_j &= \frac{1}{m} \sum_{i=0}^{m-1} (x^{(i)}_j - \mu_j)^2  \tag{6}
\end{align}
$$

>**หมายเหตุการใช้งาน:** เมื่อทำให้คุณสมบัติเป็นมาตรฐาน สิ่งสำคัญคือ
ต้องเก็บค่าที่ใช้สำหรับการทำให้เป็นมาตรฐาน - ค่าเฉลี่ยและส่วนเบี่ยงเบนมาตรฐานที่ใช้สำหรับการคำนวณ หลังจากเรียนรู้พารามิเตอร์
จากโมเดล เรามักต้องการทำนายราคาบ้านที่เราไม่เคยเห็นมาก่อน เมื่อได้รับค่า x ใหม่ (พื้นที่ห้องนั่งเล่นและจำนวนห้องนอน) เราต้องทำให้ x เป็นมาตรฐานโดยใช้ค่าเฉลี่ยและส่วนเบี่ยงเบนมาตรฐานที่เราคำนวณได้ก่อนหน้านี้จากชุดการฝึกอบรม

**การใช้งาน**

In [ ]:
def zscore_normalize_features(X):
    """
    computes  X, zcore normalized by column
    
    Args:
      X (ndarray (m,n))     : input data, m examples, n features
      
    Returns:
      X_norm (ndarray (m,n)): input normalized by column
      mu (ndarray (n,))     : mean of each feature
      sigma (ndarray (n,))  : standard deviation of each feature
    """
    # find the mean of each column/feature
    mu     = np.mean(X, axis=0)                 # mu will have shape (n,)
    # find the standard deviation of each column/feature
    sigma  = np.std(X, axis=0)                  # sigma will have shape (n,)
    # element-wise, subtract mu for that column from each example, divide by std for that column
    X_norm = (X - mu) / sigma      

    return (X_norm, mu, sigma)
 
#check our work
#from sklearn.preprocessing import scale
#scale(X_orig, axis=0, with_mean=True, with_std=True, copy=True)

มาดูขั้นตอนที่เกี่ยวข้องกับการทำให้เป็นมาตรฐาน Z กัน ภาพด้านล่างแสดงการแปลงแบบทีละขั้นตอน

In [ ]:
mu     = np.mean(X_train,axis=0)   
sigma  = np.std(X_train,axis=0) 
X_mean = (X_train - mu)
X_norm = (X_train - mu)/sigma      

fig,ax=plt.subplots(1, 3, figsize=(12, 3))
ax[0].scatter(X_train[:,0], X_train[:,3])
ax[0].set_xlabel(X_features[0]); ax[0].set_ylabel(X_features[3]);
ax[0].set_title("unnormalized")
ax[0].axis('equal')

ax[1].scatter(X_mean[:,0], X_mean[:,3])
ax[1].set_xlabel(X_features[0]); ax[0].set_ylabel(X_features[3]);
ax[1].set_title(r"X - $\mu$")
ax[1].axis('equal')

ax[2].scatter(X_norm[:,0], X_norm[:,3])
ax[2].set_xlabel(X_features[0]); ax[0].set_ylabel(X_features[3]);
ax[2].set_title(r"Z-score normalized")
ax[2].axis('equal')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
fig.suptitle("distribution of features before, during, after normalization")
plt.show()

กราฟข้างต้นแสดงความสัมพันธ์ระหว่างสองพารามิเตอร์ของชุดการฝึก "อายุ" และ "ขนาด (ตารางฟุต)" *ทั้งสองพารามิเตอร์นี้มีสเกลเท่ากัน*

- ซ้าย: Unnormalized: ช่วงของค่าหรือความแปรปรวนของลักษณะเฉพาะ 'ขนาด (ตารางฟุต)' มีขนาดใหญ่กว่าลักษณะอายุ
- ตรงกลาง: ขั้นตอนแรกคือการลบค่าเฉลี่ยหรือค่าเฉลี่ยออกจากแต่ละคุณลักษณะ ซึ่งจะทำให้คุณลักษณะเหล่านี้มีศูนย์กลางอยู่ที่ศูนย์ เป็นการยากที่จะเห็นความแตกต่างสำหรับคุณลักษณะ 'อายุ' แต่ 'ขนาด (ตารางฟุต)' มีศูนย์กลางอยู่ที่ศูนย์อย่างชัดเจน
- ขวา: ขั้นตอนที่สองคือการหารด้วยส่วนเบี่ยงเบนมาตรฐาน ซึ่งทำให้ทั้งสองลักษณะเฉพาะมีจุดศูนย์กลางอยู่ที่ศูนย์และมีขนาดที่คล้ายกัน.

เรามาทำให้ข้อมูลเป็นมาตรฐานและเปรียบเทียบกับข้อมูลเดิมกัน

In [ ]:
# normalize the original features
X_norm, X_mu, X_sigma = zscore_normalize_features(X_train)
print(f"X_mu = {X_mu}, \nX_sigma = {X_sigma}")
print(f"Peak to Peak range by column in Raw        X:{np.ptp(X_train,axis=0)}")   
print(f"Peak to Peak range by column in Normalized X:{np.ptp(X_norm,axis=0)}")

ช่วงสูงสุดถึงต่ำสุดของแต่ละคอลัมน์ถูกปรับลดจากหลายพันเท่าลงเหลือ 2-3 เท่า โดยการทำให้เป็นมาตรฐาน

In [ ]:
fig,ax=plt.subplots(1, 4, figsize=(12, 3))
for i in range(len(ax)):
    norm_plot(ax[i],X_train[:,i],)
    ax[i].set_xlabel(X_features[i])
ax[0].set_ylabel("count");
fig.suptitle("distribution of features before normalization")
plt.show()
fig,ax=plt.subplots(1,4,figsize=(12,3))
for i in range(len(ax)):
    norm_plot(ax[i],X_norm[:,i],)
    ax[i].set_xlabel(X_features[i])
ax[0].set_ylabel("count"); 
fig.suptitle("distribution of features after normalization")

plt.show()

สังเกตข้างต้น ช่วงของข้อมูลที่ปรับมาตรฐาน (แกน x) อยู่ตรงกลางที่ศูนย์และประมาณ +/- 2 ที่สำคัญที่สุด ช่วงของข้อมูลจะคล้ายกันสำหรับแต่ละลักษณะเฉพาะ

มาลองรันอัลกอริทึมการไล่ระดับลงมาของเราอีกครั้งด้วยข้อมูลที่ถูกทำให้เป็นมาตรฐานกันเถอะ. **สังเกตค่าที่ใหญ่ขึ้นอย่างมากของ alpha**. นี่จะทำให้การไล่ระดับลงมาเร็วขึ้น

In [ ]:
w_norm, b_norm, hist = run_gradient_descent(X_norm, y_train, 1000, 1.0e-1, )

คุณสมบัติที่ปรับขนาดได้ให้ผลลัพธ์ที่แม่นยำมาก **เร็วขึ้นมาก ๆ!** สังเกตการไล่ระดับของแต่ละพารามิเตอร์มีขนาดเล็กมากเมื่อสิ้นสุดการรันที่ค่อนข้างสั้น อัตราการเรียนรู้ 0.1 เป็นจุดเริ่มต้นที่ดีสำหรับการถดถอยด้วยคุณสมบัติที่ทำให้เป็นมาตรฐาน

มาพล็อตการคาดการณ์ของเรากับค่าเป้าหมายกัน โปรดทราบว่าการทำนายนั้นทำโดยใช้คุณสมบัติที่ทำให้เป็นมาตรฐาน ในขณะที่พล็อตจะแสดงโดยใช้ค่าคุณสมบัติเดิม

In [ ]:
#predict target using normalized features
m = X_norm.shape[0]
yp = np.zeros(m)
for i in range(m):
    yp[i] = np.dot(X_norm[i], w_norm) + b_norm

    # plot predictions and targets versus original features    
fig,ax=plt.subplots(1,4,figsize=(12, 3),sharey=True)
for i in range(len(ax)):
    ax[i].scatter(X_train[:,i],y_train, label = 'target')
    ax[i].set_xlabel(X_features[i])
    ax[i].scatter(X_train[:,i],yp,color=dlc["dlorange"], label = 'predict')
ax[0].set_ylabel("Price"); ax[0].legend();
fig.suptitle("target versus prediction using z-score normalized model")
plt.show()

ผลลัพธ์ดูดี มีข้อสังเกตเล็กน้อย:

- เมื่อมีหลายคุณลักษณะ เราไม่สามารถใช้พล็อตเดียวเพื่อแสดงผลลัพธ์เทียบกับคุณลักษณะได้อีกต่อไป
- ในการสร้างพล็อต ได้ใช้อินพุตที่ถูกปรับมาตรฐานแล้ว การทำนายใด ๆ ที่ใช้พารามิเตอร์ที่เรียนรู้จากชุดข้อมูลฝึกอบรมที่ถูกปรับมาตรฐานแล้ว จะต้องถูกปรับมาตรฐานด้วยเช่นกัน

**การทำนาย**

จุดมุ่งหมายของการสร้างโมเดลของเราคือเพื่อใช้ในการทำนายราคาบ้านที่ไม่อยู่ในชุดข้อมูลเดิม.
มาทำนายราคของบ้านที่มีพื้นที่ 1200 ตารางฟุต, ห้องนอน 3 ห้อง, 1 ชั้น, อายุ 40 ปีกัน.

โปรดจำไว้ว่า คุณต้องทำให้ข้อมูลเป็นมาตรฐานด้วยค่าเฉลี่ยและค่าเบี่ยงเบนมาตรฐานที่ได้มาเมื่อทำการปรับมาตรฐานข้อมูลฝึกสอน.

In [ ]:
# First, normalize out example.
x_house = np.array([1200, 3, 1, 40])
x_house_norm = (x_house - X_mu) / X_sigma
print(x_house_norm)
x_house_predict = np.dot(x_house_norm, w_norm) + b_norm
print(f" predicted price of a house with 1200 sqft, 3 bedrooms, 1 floor, 40 years old = ${x_house_predict*1000:0.0f}")

**Cost Contours**  
<img align="left" src="./images/C1_W2_Lab06_contours_v1.png"   style="width:240px;" >อีกวิธีหนึ่งในการมองการปรับขนาดคุณลักษณะคือในแง่ของเส้นโค้งค่าใช้จ่าย เมื่อขนาดคุณลักษณะไม่ตรงกัน แผนผังค่าใช้จ่ายเทียบกับพารามิเตอร์ในแผนผังเส้นโค้งจะไม่สมมาตร

ในพล็อตด้านล่าง มาตราส่วนของพารามิเตอร์จะตรงกันพล็อตด้านซ้ายคือพล็อตเส้นโค้งค่าใช้จ่ายของ w[0] ซึ่งเป็นพื้นที่ตารางฟุตเทียบกับ w[1] ซึ่งเป็นจำนวนห้องนอนก่อนการปรับมาตรฐานคุณลักษณะ
พล็อตนี้ไม่สมมาตรมาก จึงมองไม่เห็นเส้นโค้งที่เติมเต็มเส้นโค้งค่าใช้จ่าย
ตรงกันข้าม เมื่อปรับมาตรฐานคุณลักษณะ เส้นโค้งค่าใช้จ่ายจะสมมาตรมากขึ้น
ผลลัพธ์คือการอัปเดตพารามิเตอร์ระหว่างการไล่ระดับจะทำให้เกิดความก้าวหน้าเท่ากันสำหรับพารามิเตอร์แต่ละตัว




In [ ]:
plt_equal_scale(X_train, X_norm, y_train)


## ขอแสดงความยินดี!

ในห้องปฏิบัติการนี้ คุณได้:
- ใช้ฟังก์ชันการถดถอยเชิงเส้นหลายตัวแปรที่คุณพัฒนาในห้องปฏิบัติการก่อนหน้านี้
- สำรวจผลกระทบของอัตราการเรียนรู้ α ต่อการลู่เข้า
- ค้นพบประโยชน์ของการปรับขนาดคุณลักษณะโดยใช้การทำให้เป็นมาตรฐาน z-score ในการเร่งการลู่เข้า

## คำขอบคุณ
ข้อมูลที่อยู่อาศัยนี้มาจากชุดข้อมูล [Ames Housing dataset](http://jse.amstat.org/v19n3/decock.pdf) ที่รวบรวมโดย Dean De Cock เพื่อใช้ในการศึกษาศาสตร์ข้อมูล